In [6]:
import torch
print(torch.__version__)
import sys
print(sys.version)

1.9.0.post2
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


In [7]:
def subidx(n:int, remove:int) -> list:
    out = list(range(n))
    out.remove(remove)
    return out

def submatrix(A:torch.tensor, j:int) -> torch.tensor:
    n = A.shape[0]
    s = subidx(n, j)
    o = A[s, :]
    o = o[:, s]
    return o

def get_eigenvector_val_old(hermitian_matrix, i, j, disable_complex=False):
    # Old way
    lam, v = torch.linalg.eig(hermitian_matrix)
    if disable_complex:
        old_eigenvector_ij = torch.abs(v[j,i]**2)
    else:
        old_eigenvector_ij = v[j,i]**2
    
    return old_eigenvector_ij

def get_eigenvector_val(hermitian_matrix, i, j, disable_complex=False):
    # Only need to calculate eigenvalues
    lam  = torch.linalg.eigvals(hermitian_matrix)
    
    n = len(lam)
    M = submatrix(hermitian_matrix, j)
    # Only need to calculate eigenvalues
    lam_submatrix = torch.linalg.eigvals(M)

    # Left side of equation 2
    left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
    # Right side of equation 2
    right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))
    # Right divided by left
    eigenvector_ij = right / left
    
    if disable_complex:
        eigenvector_ij = torch.abs(eigenvector_ij)
    
    return eigenvector_ij

In [5]:
'''
Run with Complex Numbers
'''
# Random square matrix
rand_square_matrix = torch.rand(50, 50, dtype=float)

# Hermitian matrix
hermitian_matrix = rand_square_matrix * rand_square_matrix.T

# Old 
old_eigenvector_ij = get_eigenvector_val_old(hermitian_matrix, i=0, j=0)

# New
new_eigenvector_ij = get_eigenvector_val(hermitian_matrix, i=0, j=0)

print(f'Old Method Eigenvector ij: {old_eigenvector_ij}')
print('-'*50)
print(f'New Method Eigenvector ij: {new_eigenvector_ij}')

Old Method Eigenvector ij: (0.02086203929382368-0j)
--------------------------------------------------
New Method Eigenvector ij: (0.020862039293826073+0j)


In [8]:
'''
Run with Complex Numbers Disabled
'''
# Random square matrix
rand_square_matrix = torch.rand(50, 50, dtype=float)

# Hermitian matrix
hermitian_matrix = rand_square_matrix * rand_square_matrix.T

# Old 
old_eigenvector_ij = get_eigenvector_val_old(hermitian_matrix, i=0, j=0, disable_complex=True)

# New
new_eigenvector_ij = get_eigenvector_val(hermitian_matrix, i=0, j=0, disable_complex=True)

print(f'Old Method Eigenvector ij: {old_eigenvector_ij}')
print('-'*50)
print(f'New Method Eigenvector ij: {new_eigenvector_ij}')

Old Method Eigenvector ij: 0.0225447410369363
--------------------------------------------------
New Method Eigenvector ij: 0.02254474103693581
